<a href="https://colab.research.google.com/github/Pranesh-VM/Phishing_website_detection_CIP/blob/main/CIP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd

# Load dataset from GitHub
url = "https://raw.githubusercontent.com/Pranesh-VM/Phishing_website_detection_CIP/main/verified_online.csv"
df = pd.read_csv(url)
de=pd.read_csv("phishing_site_urls.csv")




In [8]:
#preprocessing

# Select and rename relevant columns
phishing_urls = de[['URL']].copy()
phishing_urls['Label'] = 'phishing'  # Standardize label

legitimate_urls = df[['url']].copy()
legitimate_urls.rename(columns={'url': 'URL'}, inplace=True)
legitimate_urls['Label'] = 'legitimate'  # Standardize label

# Combine datasets
combined_df = pd.concat([phishing_urls, legitimate_urls], ignore_index=True)

# Remove duplicates
combined_df.drop_duplicates(subset=['URL'], inplace=True)

# Save the preprocessed dataset
preprocessed_file = "preprocessed_urls.csv"
combined_df.to_csv(preprocessed_file, index=False)

print(f"Preprocessed dataset saved to: {preprocessed_file}")

# Display dataset info
display(combined_df.info())
display(combined_df.head())

Preprocessed dataset saved to: preprocessed_urls.csv
<class 'pandas.core.frame.DataFrame'>
Index: 570283 entries, 0 to 612435
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   URL     570283 non-null  object
 1   Label   570283 non-null  object
dtypes: object(2)
memory usage: 13.1+ MB


None

,URL,Label
0,nobell.it/70ffb52d079109dca5664cce6f317373782/...,phishing
1,www.dghjdgf.com/paypal.co.uk/cycgi-bin/webscrc...,phishing
2,serviciosbys.com/paypal.cgi.bin.get-into.herf....,phishing
3,mail.printakid.com/www.online.americanexpress....,phishing
4,thewhiskeydregs.com/wp-content/themes/widescre...,phishing
